In [12]:
def eval_logloss(y_true, y_prob):                   #评测函数
    temp=pd.DataFrame({'true':y_true,'prob':y_prob})
    sum=0
    for ind in temp.index:
        sum+=temp.loc[ind,'true']*np.log(temp.loc[ind,'prob'])+(1-temp.loc[ind,'true'])*np.log((1-temp.loc[ind,'prob']))
    return -(sum/len(temp))
def training(model,test,K,X,y,train_all,fortest=None,testresult=None):               #训练函数
    from sklearn.model_selection import StratifiedKFold
    skf=StratifiedKFold(n_splits = K,random_state = 1000,shuffle = True)
    y_test_pred=0
    y_valid_pred=0*y
    train_test_pred=0
    fit_model=[]
    for i,(train_index,test_index) in enumerate(skf.split(X,y)):
        y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
        x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
        print('Flod:',i,end=' ')
        fitted_model=model.fit(x_train, y_train)#,categorical_feature=['user_gender_id']
        pred= fitted_model.predict_proba(x_valid)[:,1]
        y_valid_pred.iloc[test_index]=pred
        if train_all==False:
            train_test_pred+=fitted_model.predict_proba(fortest)[:,1]
        y_test_pred+=fitted_model.predict_proba(test)[:,1]
        fit_model.append(fitted_model)
        import gc
        del y_train,y_valid,x_train,x_valid
        gc.collect()
    print('logloss for full training set:',eval_logloss(y,y_valid_pred),end=' ')
    if train_all==False:
        train_test_pred/=K
        print('logloss for full testing set:',eval_logloss(testresult,train_test_pred))
    return y_test_pred/K,y_valid_pred
def preproc(data):                         #对一些列进行加和除，例如总销售额，购买次数/展示次数
    data['add_shop']=data['shop_score_service']+data['shop_score_delivery']+data['shop_score_description']+data['shop_review_positive_rate']
    data['star_rshop']=data['shop_review_positive_rate']*data['shop_star_level']
    data['total_sales']=data['item_price_level']*data['item_sales_level']
    data['item_cv_pro']=data['item_collected_level']/data['item_pv_level']
    data['item_sv_pro']=data['item_sales_level']/data['item_pv_level']
    data.drop(['shop_score_description','shop_score_delivery','shop_score_service','shop_review_positive_rate',\
               'shop_star_level'],axis=1,inplace=True)
    return data
def category(tr,test):
    # 类别形整数变量转换为类别变量
    col_cato=list(tr.columns[tr.columns.str[-2:]=='id'])
    h=['instance_id','user_id','item_id','context_id','shop_id','item_brand_id','item_city_id']
    for i in h:
        if i in col_cato:
            col_cato.remove(i)
    combine=[tr,test]
    def int_to_category(df,col_cato):
        for col in col_cato:
            df[col]=df[col].astype('category')
    for data in combine:
        data=int_to_category(data,col_cato)
    combine=[tr,test]
    #类别形变量转换为指示性变量
    for col in col_cato:
        new_dummies=pd.get_dummies(tr[col],prefix=col)
        tr=pd.concat([tr,new_dummies],axis=1)
        tr.drop([col],axis=1,inplace=True)
    for col in col_cato:
        new_dummies=pd.get_dummies(test[col],prefix=col)
        test=pd.concat([test,new_dummies],axis=1)
        test.drop([col],axis=1,inplace=True)
    print(len(tr.columns))
    print(len(test.columns))
    return tr,test

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import gc
import lightgbm as lgb
train=pd.read_csv(r'D:\Data\IJCAI\train_mispret.csv')
test=pd.read_csv(r'D:\Data\IJCAI\test_mispret.csv')
new=pd.read_csv(r'D:\Data\IJCAI\test.csv')
ttest=test[['instance_id','item_id','user_id']].copy()
testo=test.copy()
K=3

In [17]:
train_all=False   #不分出测试集全部用来训练
test13=False      #测试集是否进行1/3抽样
col=['com_pre_cate', 'com_pre_pro','com_pre_cateprob', 'com_pre_proprob','num_click','num_conversion', 'num_conversiontd', 'num_show', 'item_time_diff', 'day_item_diff', 'day_user_diff','is_trade','context_timestamp','context_id', 'user_id', 'item_id', 'shop_id', 'shop_score_delivery', 'shop_score_description', 'item_brand_id', 'item_sales_level', 'shop_score_service', 'shop_review_positive_rate', 'user_age_level', 'user_star_level', 'item_city_id', 'item_pv_level', 'shop_review_num_level', 'item_collected_level', 'item_category_list_0', 'context_page_id', 'item_price_level', 'user_occupation_id', 'user_gender_id', 'shop_star_level', 'predict_category_property_7908382889764677758', 'predict_category_property_8277336076276184272', 'predict_category_property_5799347067982556520', 'predict_category_property_509660095530134768', 'predict_category_property_7822717283490579102', 'predict_category_property_4144951645589726964', 'predict_category_property_7258015885215914736', 'predict_category_property_7492960463130085436', 'predict_category_property_2356297995131360540', 'predict_category_property_6007511169620956394', 'item_category_list_1', 'predict_category_property_2011981573061447208', 'predict_category_property_1950314698730389427', 'predict_category_property_5755694407684602296', 'predict_category_property_4864561375761257861', 'predict_category_property_6731593623692005614', 'predict_category_property_836752724084922533', 'predict_category_property_8710739180200009128', 'predict_category_property_7314150500379498593', 'predict_category_property_1771349742445680985', 'predict_category_property_6948560137918828076', 'predict_category_property_8426409705041090944', 'predict_category_property_1760164811125093110', 'predict_category_property_8896700187874717254', 'predict_category_property_4879721024980945592', 'predict_category_property_8257512457089702259', 'predict_category_property_2436715285093487584', 'predict_category_property_5912343598847406450', 'predict_category_property_2482352103497268539', 'predict_category_property_7730907179336883505']
# col=['num_click', 'num_conversion', 'num_conversiontd', 'num_show', 'item_time_diff', 'day_item_diff', 'day_user_diff','is_trade','context_timestamp','context_id', 'user_id', 'item_id', 'shop_id', 'shop_score_delivery', 'shop_score_description', 'item_brand_id', 'item_sales_level', 'shop_score_service', 'shop_review_positive_rate', 'user_age_level', 'user_star_level', 'item_city_id', 'item_pv_level', 'shop_review_num_level', 'item_collected_level', 'item_category_list_0', 'context_page_id', 'item_price_level', 'user_occupation_id', 'user_gender_id', 'shop_star_level', 'predict_category_property_7908382889764677758', 'predict_category_property_8277336076276184272', 'predict_category_property_5799347067982556520', 'predict_category_property_509660095530134768', 'predict_category_property_7822717283490579102', 'predict_category_property_4144951645589726964', 'predict_category_property_7258015885215914736', 'predict_category_property_7492960463130085436', 'predict_category_property_2356297995131360540', 'predict_category_property_6007511169620956394', 'item_category_list_1', 'predict_category_property_2011981573061447208', 'predict_category_property_1950314698730389427', 'predict_category_property_5755694407684602296', 'predict_category_property_4864561375761257861', 'predict_category_property_6731593623692005614', 'predict_category_property_836752724084922533', 'predict_category_property_8710739180200009128', 'predict_category_property_7314150500379498593', 'predict_category_property_1771349742445680985', 'predict_category_property_6948560137918828076', 'predict_category_property_8426409705041090944', 'predict_category_property_1760164811125093110', 'predict_category_property_8896700187874717254', 'predict_category_property_4879721024980945592', 'predict_category_property_8257512457089702259', 'predict_category_property_2436715285093487584', 'predict_category_property_5912343598847406450', 'predict_category_property_2482352103497268539', 'predict_category_property_7730907179336883505']
# col=['item_id','item_property_list','item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id', 'context_timestamp', 'context_page_id', 'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description', 'is_trade', 'item_category_list_dup']
train['timestamp']=pd.to_datetime(train['context_timestamp'].copy(),unit='s')
train['day']=train['timestamp'].copy().dt.day
train=train[train['day']>17]
tr=train[col].copy()
tr=preproc(tr)
col.remove('is_trade')
test=testo.copy()
test=test[col].copy()
test=preproc(test)
# tr,test=category(tr,test)
if train_all==False:                                      #是否分出测试集
    fortest=tr[tr['context_timestamp']>1537718399]
    np.random.seed(1000)
    if test13==True:                #验证集进行1/3抽样
        lis=[]
        for i in fortest.index:
            if np.random.randint(3)==1:
                lis.append(i)
        fortest=fortest.loc[lis]
    testresult=fortest['is_trade'].copy()
    fortest.drop(['is_trade'],axis=1,inplace=True)
    fortrain=tr[tr['context_timestamp']<=1537718399]
    X=fortrain.drop(['is_trade'],axis=1)
    y=fortrain['is_trade']
else:
    X=tr.drop(['is_trade'],axis=1)
    y=tr['is_trade']
# combine=[fortest,X,test]
# for data in combine:
#     data['timestamp']=pd.to_datetime(data['context_timestamp'].copy(),unit='s')
#     data['weekday']=data['timestamp'].copy().dt.weekday
#     data['hour']=data['timestamp'].copy().dt.hour
#     data['day']=data['timestamp'].copy().dt.day
#     data['minute']=data['timestamp'].copy().dt.minute
# #     data['second']=data['timestamp'].copy().dt.second
#     data['tm_hour']=data['hour'].copy()+data['minute'].copy()/60.0
#     data.drop(['timestamp','context_timestamp'],axis=1,inplace=True)
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits = K,random_state = 1000,shuffle = True)   
model=lgb.LGBMClassifier(learning_rate=0.08,num_leaves=60, max_depth=7, n_estimators=80,min_child_samples=50,n_jobs=20,reg_lambda=1)
y_test_pred,y_valid_pred=training(model,test,K,X,y,train_all,fortest,testresult)

Flod: 0 Flod: 1 Flod: 2 Flod: 3 Flod: 4 Flod: 5 logloss for full training set: 0.08798155057818735 

no                   0.08318013595769923              0.09108811958036114<br/>
0.08913559182213175 logloss for full testing set: 0.08095946639289714<br/>
0.08983365980428494      0.0819244060584064 &nbsp;&nbsp;&nbsp;&nbsp;去掉17号&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;num_conversion<br/>

In [ ]:
# importances=model.feature_importances_
# indices=np.argsort(-importances)
# print(importances[indices][:54])
# print(X.columns[indices][:54])
# list(X.columns[indices][:54])

In [18]:
train_all=True   #不分出测试集全部用来训练
col=['com_pre_cateprob', 'com_pre_proprob','num_click','num_conversion', 'num_conversiontd', 'num_show', 'item_time_diff', 'day_item_diff', 'day_user_diff','is_trade','context_timestamp','context_id', 'user_id', 'item_id', 'shop_id', 'shop_score_delivery', 'shop_score_description', 'item_brand_id', 'item_sales_level', 'shop_score_service', 'shop_review_positive_rate', 'user_age_level', 'user_star_level', 'item_city_id', 'item_pv_level', 'shop_review_num_level', 'item_collected_level', 'item_category_list_0', 'context_page_id', 'item_price_level', 'user_occupation_id', 'user_gender_id', 'shop_star_level', 'predict_category_property_7908382889764677758', 'predict_category_property_8277336076276184272', 'predict_category_property_5799347067982556520', 'predict_category_property_509660095530134768', 'predict_category_property_7822717283490579102', 'predict_category_property_4144951645589726964', 'predict_category_property_7258015885215914736', 'predict_category_property_7492960463130085436', 'predict_category_property_2356297995131360540', 'predict_category_property_6007511169620956394', 'item_category_list_1', 'predict_category_property_2011981573061447208', 'predict_category_property_1950314698730389427', 'predict_category_property_5755694407684602296', 'predict_category_property_4864561375761257861', 'predict_category_property_6731593623692005614', 'predict_category_property_836752724084922533', 'predict_category_property_8710739180200009128', 'predict_category_property_7314150500379498593', 'predict_category_property_1771349742445680985', 'predict_category_property_6948560137918828076', 'predict_category_property_8426409705041090944', 'predict_category_property_1760164811125093110', 'predict_category_property_8896700187874717254', 'predict_category_property_4879721024980945592', 'predict_category_property_8257512457089702259', 'predict_category_property_2436715285093487584', 'predict_category_property_5912343598847406450', 'predict_category_property_2482352103497268539', 'predict_category_property_7730907179336883505']
train['timestamp']=pd.to_datetime(train['context_timestamp'].copy(),unit='s')
train['day']=train['timestamp'].copy().dt.day
tr=train[train['day']>17]
tr=train[col].copy()
tr=preproc(tr)
col.remove('is_trade')
test=testo.copy()
test=test[col].copy()
test=preproc(test)
if train_all==False:
    fortest=tr[tr['context_timestamp']>1537718399]
    testresult=fortest['is_trade'].copy()
    fortest.drop(['is_trade'],axis=1,inplace=True)
    fortrain=tr[tr['context_timestamp']<=1537718399]
    X=fortrain.drop(['is_trade'],axis=1)
    y=fortrain['is_trade']
else:
    X=tr.drop(['is_trade'],axis=1)
    y=tr['is_trade']
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits = K,random_state = 1000,shuffle = True)   
import xgboost as xgb
model=xgb.XGBClassifier(n_estimators=200,max_depth=6,objective="binary:logistic",learning_rate=0.09,subsample=0.8,min_child_weight=8,\
                        colsample_bytree=0.9,scale_pos_weight=1.6,gamma=8,seed=100,reg_alpha=8,reg_lambda=1.3,nthread=19)
y_test_pred1,y_valid_pred=training(model,test,K,X,y,train_all,fortest,testresult)
# 0.0910413419281298 logloss for full testing set: 0.08102993576771603  600  6 0.09

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Flod: 0 Flod: 1 Flod: 2 Flod: 3 Flod: 4 Flod: 5 logloss for full training set: 0.08970881260168051 

In [19]:
if train_all==True:
    ttest['predicted_score']=(y_test_pred+y_test_pred1)/2

In [20]:
if train_all==True:
    ttest[['instance_id','predicted_score']].to_csv(r'D:\Data\IJCAI\model\submission.csv',sep=' ',float_format='%.6f',index=False)